In [1]:
# --- 1. INSTALL ONLY WHAT WE NEED ---
!pip install tf-keras keras-cv-attention-models -q

# --- 2. THE "MAGIC" PATCH (Your Code) ---
import os
import sys

# Force Legacy Keras (Keras 2)
os.environ["TF_USE_LEGACY_KERAS"] = "1"

import tf_keras

# Redirect standard Keras calls to tf_keras (The Hot-Swap)
sys.modules["keras"] = tf_keras
sys.modules["tensorflow.keras"] = tf_keras
sys.modules["keras.api"] = tf_keras # Safety measure for newer TF versions
sys.modules["keras.api._v2"] = tf_keras # Safety measure

import tensorflow as tf
import numpy as np
import pandas as pd
import keras_cv_attention_models

# Redirect tf.keras explicitly
tf.keras = tf_keras

print(f"✅ TensorFlow Version: {tf.__version__}")
print(f"✅ Keras Version: {tf.keras.__version__} (Should be ~2.15 or 2.16)")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.6/802.6 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.8 MB/s eta 0:00:00
✅ TensorFlow Version: 2.19.0
✅ Keras Version: 2.19.0 (Should be ~2.15 or 2.16)


In [2]:
# ==========================================
# PART 1: MAXVIT GENERATOR (Legacy Keras) -> SAVES TO DRIVE
# ==========================================
import os
import sys

# 1. FORCE LEGACY KERAS
os.environ["TF_USE_LEGACY_KERAS"] = "1"

# 2. INSTALL DEPENDENCIES
!pip install tf-keras keras-cv-attention-models -q

import tf_keras
import tensorflow as tf
import numpy as np
import pandas as pd
import keras_cv_attention_models
from google.colab import drive

# 3. MOUNT DRIVE
print("🔌 Mounting Google Drive...")
try:
    drive.mount('/content/drive', force_remount=True)
except:
    pass

# 4. PATHS
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
DATA_ROOT = "/content/drive/MyDrive/skincareapp/acne clean pigmentation wrinkles"
MODEL_PATH = "/content/drive/MyDrive/models/maxvit_tiny_skin_model_FINAL_TUNED.keras"
TRAIN_CSV = os.path.join(DATA_ROOT, "labels.csv")

# *** KEY CHANGE: Save to Drive ***
SAVE_PATH = os.path.join(DATA_ROOT, "maxvit_train_preds.npy")

# 5. CUSTOM OBJECTS
def weighted_bce_dummy(y_true, y_pred):
    return tf.keras.backend.binary_crossentropy(y_true, y_pred)

custom_objects = {
    "weighted_bce": weighted_bce_dummy,
    "gelu": tf.nn.gelu,
    "swish": tf.nn.swish,
    "Functional": tf_keras.models.Model
}

# 6. LOAD DATA
print("📂 Loading Dataset...")
df = pd.read_csv(TRAIN_CSV)
df["filename"] = df["filename"].apply(lambda x: os.path.join(DATA_ROOT, x))

from sklearn.model_selection import train_test_split
CLASSES = ["acne", "pigmentation", "wrinkles"]
train_val_df, test_df = train_test_split(df, test_size=0.15, random_state=42, stratify=df[CLASSES])
train_df, val_df = train_test_split(train_val_df, test_size=0.15, random_state=42, stratify=train_val_df[CLASSES])

def load_image(path):
    img = tf.io.read_file(path)
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, IMG_SIZE)
    return img

print(f"📊 Preparing {len(train_df)} images for MaxViT...")
train_ds = tf.data.Dataset.from_tensor_slices(train_df["filename"].values)
train_ds = train_ds.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# 7. PREDICT & SAVE
if os.path.exists(SAVE_PATH):
    print(f"⚠️ File {SAVE_PATH} already exists! Skipping generation.")
else:
    print("🤖 Loading MaxViT Model...")
    try:
        model = tf_keras.models.load_model(MODEL_PATH, custom_objects=custom_objects, compile=False)
        print("✅ MaxViT Loaded! Generating predictions...")
        preds = model.predict(train_ds, verbose=1)
        np.save(SAVE_PATH, preds)
        print(f"🎉 PART 1 DONE: Saved to '{SAVE_PATH}'")
    except Exception as e:
        print(f"\n❌ MaxViT Error: {e}")

🔌 Mounting Google Drive...
Mounted at /content/drive
📂 Loading Dataset...
📊 Preparing 3656 images for MaxViT...
🤖 Loading MaxViT Model...
✅ MaxViT Loaded! Generating predictions...
115/115 [==============================] - 605s 5s/step
🎉 PART 1 DONE: Saved to '/content/drive/MyDrive/skincareapp/acne clean pigmentation wrinkles/maxvit_train_preds.npy'


In [1]:
# ==========================================
# PART 2: EFFICIENTNET GENERATOR (Modern Keras) -> SAVES TO DRIVE
# ==========================================
import os
import sys

# 1. FORCE UNSET LEGACY MODE
if "TF_USE_LEGACY_KERAS" in os.environ:
    del os.environ["TF_USE_LEGACY_KERAS"]

# 2. UPDATE KERAS
!pip install keras --upgrade -q

import keras
import tensorflow as tf
import numpy as np
import pandas as pd
from google.colab import drive

# 3. MOUNT DRIVE
print("🔌 Mounting Google Drive...")
try:
    drive.mount('/content/drive', force_remount=True)
except:
    pass

# 4. PATHS
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
DATA_ROOT = "/content/drive/MyDrive/skincareapp/acne clean pigmentation wrinkles"
MODEL_PATH = "/content/drive/MyDrive/models/efficientnetv2s_se_best.keras"
TRAIN_CSV = os.path.join(DATA_ROOT, "labels.csv")

# *** KEY CHANGE: Save to Drive ***
SAVE_PATH = os.path.join(DATA_ROOT, "effnet_train_preds.npy")

# 5. CUSTOM OBJECTS
@keras.saving.register_keras_serializable()
def weighted_bce(y_true, y_pred):
    return keras.losses.binary_crossentropy(y_true, y_pred)

# 6. LOAD DATA
print("📂 Loading Dataset...")
df = pd.read_csv(TRAIN_CSV)
df["filename"] = df["filename"].apply(lambda x: os.path.join(DATA_ROOT, x))

from sklearn.model_selection import train_test_split
CLASSES = ["acne", "pigmentation", "wrinkles"]
train_val_df, test_df = train_test_split(df, test_size=0.15, random_state=42, stratify=df[CLASSES])
train_df, val_df = train_test_split(train_val_df, test_size=0.15, random_state=42, stratify=train_val_df[CLASSES])

def load_image(path):
    img = tf.io.read_file(path)
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, IMG_SIZE)
    return img

print(f"📊 Preparing {len(train_df)} images for EfficientNet...")
train_ds = tf.data.Dataset.from_tensor_slices(train_df["filename"].values)
train_ds = train_ds.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# 7. PREDICT & SAVE
if os.path.exists(SAVE_PATH):
    print(f"⚠️ File {SAVE_PATH} already exists! Skipping generation.")
else:
    print("🤖 Loading EfficientNet Model...")
    try:
        model = keras.saving.load_model(MODEL_PATH, compile=False)
        print("✅ EfficientNet Loaded! Generating predictions...")
        preds = model.predict(train_ds, verbose=1)
        np.save(SAVE_PATH, preds)
        print(f"🎉 PART 2 DONE: Saved to '{SAVE_PATH}'")
    except Exception as e:
        print(f"\n❌ EfficientNet Error: {e}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 22.1 MB/s eta 0:00:00
🔌 Mounting Google Drive...
Mounted at /content/drive
📂 Loading Dataset...
📊 Preparing 3656 images for EfficientNet...
🤖 Loading EfficientNet Model...
✅ EfficientNet Loaded! Generating predictions...
115/115 ━━━━━━━━━━━━━━━━━━━━ 676s 6s/step
🎉 PART 2 DONE: Saved to '/content/drive/MyDrive/skincareapp/acne clean pigmentation wrinkles/effnet_train_preds.npy'


In [2]:
import numpy as np
import os
from google.colab import drive

# Ensure Drive is accessible
drive.mount('/content/drive', force_remount=True)

DATA_ROOT = "/content/drive/MyDrive/skincareapp/acne clean pigmentation wrinkles"

# 1. PATHS TO LOAD FROM DRIVE
maxvit_path = os.path.join(DATA_ROOT, "maxvit_train_preds.npy")
effnet_path = os.path.join(DATA_ROOT, "effnet_train_preds.npy")
save_path = "teacher_train_probs.npy" # Save locally for training (or change to DATA_ROOT to save to drive)

# 2. CHECK & MERGE
if os.path.exists(maxvit_path) and os.path.exists(effnet_path):
    print("🔄 Found both files on Drive. Merging...")

    maxvit_preds = np.load(maxvit_path)
    effnet_preds = np.load(effnet_path)

    # Ensemble Logic
    teacher_train_probs = (0.7 * maxvit_preds) + (0.3 * effnet_preds)

    np.save(save_path, teacher_train_probs)
    print(f"🏆 SUCCESS! Final soft labels saved as '{save_path}'")
    print(f"Shape: {teacher_train_probs.shape}")
    print("You can now proceed to train your MobileViT student!")
else:
    print("❌ One or both prediction files are missing from Google Drive.")
    print(f"Checking MaxViT: {maxvit_path} -> {os.path.exists(maxvit_path)}")
    print(f"Checking EffNet: {effnet_path} -> {os.path.exists(effnet_path)}")

Mounted at /content/drive
🔄 Found both files on Drive. Merging...
🏆 SUCCESS! Final soft labels saved as 'teacher_train_probs.npy'
Shape: (3656, 3)
You can now proceed to train your MobileViT student!
